# The Jacobian- and Hessian-free Halley's Method

Say we have a system of $n$ equations with $n$ unknowns

$$
f(x)=0
$$

and $f\in \mathbb R^n\to\mathbb R^n$ is sufficiently smooth.

Given a initial guess $x_0$, Halley's method specifies a series of points approximating the solution, where each iteration is

$$
x^{(i+1)}=x^{(i)}+\frac{a^{(i)}a^{(i)}}{a^{(i)}+b^{(i)}/2}
$$

where the vector multiplication and division $ab, a/b$ is defined in Banach algebra, and the vectors $a^{(i)}, b^{(i)}$ are defined as

$$
J(x^{(i)})a^{(i)} = -f(x^{(i)})
$$

and

$$
J(x^{(i)})b^{(i)} = H(x^{(i)})a^{(i)}a^{(i)}
$$


The full Jacobian (a matrix) and full Hessian (a 3-tensor) representation can be avoided by using forward-mode automatic differentiation. It is well known that a forward evaluation on a dual number $(x, v)$ gives the Jacobian-vector product,

$$
f(x,v)=(f(x),Jv)
$$

and similarly a forward evaluation on a second order Taylor expansion gives the Hessian-vector-vector product,

$$
f(x,v,0)=f(x,Jv,Hvv)
$$

Below, we demonstrate this possibility with TaylorDiff.jl.

## Jacobian-free Newton Krylov

To get started we first get familiar with the JFNK:

In [1]:
# The Jacobi- and Hessian-free Halley method for solving nonlinear equations

using TaylorDiff
using LinearAlgebra
using LinearSolve

function newton(f, x0, p; tol=1e-10, maxiter=100)
    x = x0
    for i in 1:maxiter
        fx = f(x, p)
        error = norm(fx)
        println("Iteration $i: x = $x, f(x) = $fx, error = $error")
        if error < tol
            return x
        end
        get_derivative = (v, u, a, b) -> v .= derivative(x -> f(x, p), x, u, 1)
        operator = FunctionOperator(get_derivative, similar(x), similar(x))
        problem = LinearProblem(operator, -fx)
        sol = solve(problem, KrylovJL_GMRES())
        x += sol.u
    end
    return x
end

newton (generic function with 1 method)

## Jacobian- and Hessian-free Halley

This naturally follows, only difference is replacing the rhs by Hessian-vector-vector product:

In [2]:
function halley(f, x0, p; tol=1e-10, maxiter=100)
    x = x0
    for i in 1:maxiter
        fx = f(x, p)
        error = norm(fx)
        println("Iteration $i: x = $x, f(x) = $fx, error = $error")
        if error < tol
            return x
        end
        get_derivative = (v, u, a, b) -> v .= derivative(x -> f(x, p), x, u, 1)
        operator = FunctionOperator(get_derivative, similar(x), similar(x))
        problem = LinearProblem(operator, -fx)
        a = solve(problem, KrylovJL_GMRES()).u
        Haa = derivative(x -> f(x, p), x, a, 2)
        problem2 = LinearProblem(operator, Haa)
        b = solve(problem2, KrylovJL_GMRES()).u
        x += (a .* a) ./ (a .+ b ./ 2)
    end
    return x
end

halley (generic function with 1 method)

In [3]:
# Testing with simple examples:

f(x, p) = x .* x - p

f (generic function with 1 method)

In [4]:
newton(f, [1., 1.], [2., 2.])

Iteration 1: x = [1.0, 1.0], f(x) = [-1.0, -1.0], error = 1.4142135623730951


Iteration 2: x = [1.5, 1.5], f(x) = [0.25, 0.25], error = 0.3535533905932738
Iteration 3: x = [1.4166666666666667, 1.4166666666666667], f(x) = [0.006944444444444642, 0.006944444444444642], error = 0.009820927516480105
Iteration 4: x = [1.4142156862745099, 1.4142156862745099], f(x) = [6.007304882871267e-6, 6.007304882871267e-6], error = 8.495612038666664e-6
Iteration 5: x = [1.4142135623746899, 1.4142135623746899], f(x) = [4.510614104447086e-12, 4.510614104447086e-12], error = 6.378971641140442e-12


2-element Vector{Float64}:
 1.4142135623746899
 1.4142135623746899

In [5]:
halley(f, [1., 1.], [2., 2.])

Iteration 1: x = [1.0, 1.0], f(x) = [-1.0, -1.0], error = 1.4142135623730951


Iteration 2: x = [1.4000000000000001, 1.4000000000000001], f(x) = [-0.03999999999999959, -0.03999999999999959], error = 0.05656854249492323
Iteration 3: x = [1.4142131979695431, 1.4142131979695431], f(x) = [-1.0306887576749801e-6, -1.0306887576749801e-6], error = 1.4576140196894333e-6
Iteration 4: x = [1.414213562373142, 1.414213562373142], f(x) = [1.3278267374516872e-13, 1.3278267374516872e-13], error = 1.877830580585795e-13


2-element Vector{Float64}:
 1.414213562373142
 1.414213562373142